# 1-Model

在当前人工智能领域, 主流大模型从架构上大致可分为稠密（Dense）模型和混合专家（Mixture of Expert, MoE）模型 。稠密模型中所有参数在每次计算时都会参与运算；混合专家模型则将不同的 “专家” 模块组合, 根据输入选择合适的专家处理, 能在保证效果的同时减少计算量和参数量.

MiniMind 系列模型在 Llama 3.1 的基础上设计, 基于经典的 Transformer Deocder-Only 架构，在这一板块, 我们将围绕 MiniMind 系列模型的源代码展开学习.

## MiniMind Dense Model

作者提供了对其 MiniMind Dense Model 模型结构的可视化：

![image](../images/LLM-structure.png)

In [7]:
import math
import torch
from torch import nn
from transformers import PretrainedConfig
from transformers.activations import ACT2FN
from typing import Optional, Tuple, List, Union
import torch.nn.functional as F
from transformers import PreTrainedModel, GenerationMixin, PretrainedConfig
from transformers.modeling_outputs import CausalLMOutputWithPast

class MiniMindConfig(PretrainedConfig):
    model_type = "minimind"

    def __init__(
            self,
            dropout: float = 0.0,
            bos_token_id: int = 1,
            eos_token_id: int = 2,
            hidden_act: str = 'silu',
            hidden_size: int = 512,
            intermediate_size: int = None,
            max_position_embeddings: int = 32768,
            num_attention_heads: int = 8,
            num_hidden_layers: int = 8,
            num_key_value_heads: int = 2,
            vocab_size: int = 6400,
            rms_norm_eps: float = 1e-05,
            rope_theta: int = 1000000.0,
            flash_attn: bool = True,
            ####################################################
            # Here are the specific configurations of MOE
            # When use_moe is false, the following is invalid
            ####################################################
            use_moe: bool = False,
            num_experts_per_tok: int = 2,
            n_routed_experts: int = 4,
            n_shared_experts: int = 1,
            scoring_func: str = 'softmax',
            aux_loss_alpha: float = 0.1,
            seq_aux: bool = True,
            norm_topk_prob: bool = True,
            **kwargs
    ):
        super().__init__(**kwargs)
        self.dropout = dropout
        self.bos_token_id = bos_token_id
        self.eos_token_id = eos_token_id
        self.hidden_act = hidden_act
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.max_position_embeddings = max_position_embeddings
        self.num_attention_heads = num_attention_heads
        self.num_hidden_layers = num_hidden_layers
        self.num_key_value_heads = num_key_value_heads
        self.vocab_size = vocab_size
        self.rms_norm_eps = rms_norm_eps
        self.rope_theta = rope_theta
        self.flash_attn = flash_attn
        ####################################################
        # Here are the specific configurations of MOE
        # When use_moe is false, the following is invalid
        ####################################################
        self.use_moe = use_moe
        self.num_experts_per_tok = num_experts_per_tok  # 每个token选择的专家数量
        self.n_routed_experts = n_routed_experts  # 总的专家数量
        self.n_shared_experts = n_shared_experts  # 共享专家
        self.scoring_func = scoring_func  # 评分函数，默认为'softmax'
        self.aux_loss_alpha = aux_loss_alpha  # 辅助损失的alpha参数
        self.seq_aux = seq_aux  # 是否在序列级别上计算辅助损失
        self.norm_topk_prob = norm_topk_prob  # 是否标准化top-k概率

### 均方根层归一化 (Root Mean Square Layer Normalization, RMSNorm)

RMSNorm 是对 LayerNorm 的一个改进,  没有做 re-center 操作（移除了均值项）, 可以看作 LayerNorm 在均值为零时的特例, 使用平方根均值归一化降低噪声影响。

- **Layer Norm**

$$y = \frac{x-E(x)}{\sqrt{Var(x) + \epsilon}} * \gamma + \beta$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), 层归一化对 embedding_dim 维度进行归一化操作, 其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢,  $\gamma$,  $\beta$ 均为可学习参数。

- **RMS Norm**

$$a_i=\frac{a_i}{RMS(a) + \epsilon} * \gamma,  \quad where \quad RMS(a) = \sqrt{\frac{1}{n}\sum^n_{i=1}a^2_i}.$$

假设输入张量形状为 (batch_size,  sequence_length,  embedding_dim), RMS Norm 对 embedding_dim 维度进行归一化,其中,  其中,  $\epsilon$ 是一个超参数, 用于防止分母为零导致结果上溢, $\gamma$ 为可学习参数.

不难发现, 当均值为零时, Layer Norm 退化为 RMS Norm. 这是因为 RMS Norm 在 Layer Norm 的基础上舍弃了中心化操作, 仅用缩放进行归一化, 其不改变数据原本的分布, 有利于激活函数输出的稳定.

In [3]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float = 1e-5):
        super().__init__()
        self.eps = eps  # 防止分母等于零导致结果上溢
        self.weight = nn.Parameter(torch.ones(dim)) # 可学习参数

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        return self.weight * self._norm(x.float()).type_as(x)

### Rotary Position Embedding, RoPE

旋转位置编码是一种能将相对位置信息集成到 self-attention 中, 进而提升 transformer 架构性能的位置编码方式, 和绝对位置编码相比, RoPE 具有很好的外推性, 是目前的主流位置编码方式.

外推性的解释, 通俗来说就是训练的时候限制了 512 的上下文长度，那么推理时如果面对超过该长度的文本，LLM 可能无法正确处理.

- **绝对位置编码**

绝对位置编码是早期 Transformer 架构采用的绝对位置编码方案，及那个每个位置映射为固定的向量表示.

$$f_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i,i)=\boldsymbol{W}_{t:t\in\{q,k,v\}}(\boldsymbol{x}_i+\boldsymbol{p}_i)$$

其中编码向量 $p_i$ 的计算使用如下公式：

$$\boldsymbol{p}_{i,2t}=\sin\left(k/1000^{2t/d}\right), \boldsymbol{p}_{i,2t+1}=\cos\left(k/1000^{2t/d}\right)$$

正如其名，绝对位置编码只考虑了输入序列中的绝对位置关系，对于 token 之间的相对信息则没有纳入考虑.

- **旋转位置编码**

假定 query 和 key 的内积操作可以被函数 g 表示，该函数 g 的输入是词嵌入向量 $x_m, x_n$ 和它们之间的相对位置 $m-n$:

$$<f_q(x_m ,m), f_k(x_n, n)>=g(x_m, x_n, m, n)$$

旋转位置编码就是找到一个使上式成立的位置编码方式. 

出于认识的目的，我们省略复杂的数学推导，直接看 RoPE 的的结论：

存在这样一个正交矩阵：

$$\boldsymbol{R}_{\Theta,m}^d=\underbrace{\begin{pmatrix}\cos m\theta_0&-\sin m\theta_0&0&0&\cdots&0&0\\\sin m\theta_0&\cos m\theta_0&0&0&\cdots&0&0\\0&0&\cos m\theta_1&-\sin m\theta_1&\cdots&0&0\\0&0&\sin m\theta_1&\cos m\theta_1&\cdots&0&0\\\vdots&\vdots&\vdots&\vdots&\ddots&\vdots&\vdots\\0&0&0&0&\cdots&\cos m\theta_{d/2-1}&-\sin m\theta_{d/2-1}&-\sin m\theta_{d/2-1}\end{pmatrix}}_{\boldsymbol{W}_m}$$

其中，$\Theta=\left\{\theta_i=10000^{-2(i-1)/d},i\in[1,2,\ldots,d/2]\right\}$

我们可以将 query 和 key 的内积操作转换为与原始向量 $x$ 相关的以下等价形式：

$$
\boldsymbol{q}_m^\mathbf{T}\boldsymbol{k}_n=\left(\boldsymbol{R}_{\Theta,m}^d\boldsymbol{W}_q\boldsymbol{x}_m\right)^\mathbf{T}\left(\boldsymbol{R}_{\Theta,n}^d\boldsymbol{W}_k\boldsymbol{x}_n\right)=\boldsymbol{x}_m^\mathbf{T}\boldsymbol{W}_q\boldsymbol{R}_{\Theta,n-m}^d\boldsymbol{W}_k\boldsymbol{x}_n
$$

其中， $\boldsymbol{R}_{\Theta,n-m}^d=\left(\boldsymbol{R}_{\Theta,m}^d\right)^\mathbf{T}\boldsymbol{R}_{\Theta,n}^d$.

由于 $\boldsymbol{R}_{\Theta,m}^d$ 的稀疏性，直接使用矩阵乘法会浪费算力，因此代码中采用下述方式实现：

$$\boldsymbol{R}_{\Theta,m}^{d}\boldsymbol{x}=\begin{pmatrix}x_{0}\\x_{1}\\x_{2}\\x_{3}\\\vdots\\x_{d-2}\\x_{d-1}\end{pmatrix}\otimes\begin{pmatrix}\cos m\theta_{0}\\\cos m\theta_{0}\\\cos m\theta_{1}\\\cos m\theta_{1}\\\vdots\\\cos m\theta_{d/2-1}\\\cos m\theta_{d/2-1}\end{pmatrix}+\begin{pmatrix}-x_{1}\\x_{0}\\-x_{3}\\x_{2}\\\vdots\\-x_{d-1}\\x_{d-2}\end{pmatrix}\otimes\begin{pmatrix}\sin m\theta_{0}\\\sin m\theta_{0}\\\sin m\theta_{1}\\\sin m\theta_{1}\\\vdots\\\sin m\theta_{d/2-1}\\\sin m\theta_{d/2-1}\end{pmatrix}
$$

In [ ]:
def precompute_freqs_cis(dim: int, end: int = int(32 * 1024), theta: float = 1e6):
    """
    预计算旋转位置编码的频率和相位。

    参数:
    - dim (int): 嵌入维度。
    - end (int): 序列长度的最大值，默认为 32 * 1024。
    - theta (float): 控制频率范围的参数，默认为 1e6。

    返回:
    - freqs_cos (torch.Tensor): 余弦频率张量，形状为 (end, dim)。
    - freqs_sin (torch.Tensor): 正弦频率张量，形状为 (end, dim)。
    """
    # 计算频率
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2)[: (dim // 2)].float() / dim))
    # 生成时间步
    t = torch.arange(end, device=freqs.device)
    # 计算频率矩阵
    freqs = torch.outer(t, freqs).float()
    # 计算余弦和正弦频率
    freqs_cos = torch.cat([torch.cos(freqs), torch.cos(freqs)], dim=-1)
    freqs_sin = torch.cat([torch.sin(freqs), torch.sin(freqs)], dim=-1)
    return freqs_cos, freqs_sin

def apply_rotary_pos_emb(q, k, cos, sin, position_ids=None, unsqueeze_dim=1):
    """
    应用旋转位置编码到 Query 和 Key 向量。

    参数:
    - q (torch.Tensor): Query 向量，形状为 (..., head_dim)。
    - k (torch.Tensor): Key 向量，形状为 (..., head_dim)。
    - cos (torch.Tensor): 预计算的余弦频率张量。
    - sin (torch.Tensor): 预计算的正弦频率张量。
    - position_ids (Optional[torch.Tensor]): 位置索引，默认为 None。
    - unsqueeze_dim (int): 在余弦和正弦频率张量中扩展维度的索引，默认为 1。

    返回:
    - q_embed (torch.Tensor): 应用旋转位置编码后的 Query 向量。
    - k_embed (torch.Tensor): 应用旋转位置编码后的 Key 向量。
    """
    def rotate_half(x):
        """
        对张量进行旋转操作，将后一半的值取负并与前一半交换位置。

        参数:
        - x (torch.Tensor): 输入张量。

        返回:
        - torch.Tensor: 旋转后的张量。
        """
        return torch.cat((-x[..., x.shape[-1] // 2:], x[..., : x.shape[-1] // 2]), dim=-1)

    # 对 Query 和 Key 应用旋转位置编码
    q_embed = (q * cos.unsqueeze(unsqueeze_dim)) + (rotate_half(q) * sin.unsqueeze(unsqueeze_dim))
    k_embed = (k * cos.unsqueeze(unsqueeze_dim)) + (rotate_half(k) * sin.unsqueeze(unsqueeze_dim))
    return q_embed, k_embed

我们知道，RoPE 是在 Attention 阶段生成 Query 和 Key 向量后，对这两个向量进行位置编码的.

对于 MiniMindLM，嵌入维度为 512，注意力头数量为 8，故每一个注意力头的维度应该为 512 / 8 = 64.

我们使用固定形状的张量代表 Query 和 Key 向量，对 RoPE 的应用过程进行观察.

In [6]:
xq,  xk = torch.randn((2,  16,  4,  64)), torch.randn((2,  16,  4,  64))

### Attention

注意力机制（Attention Mechanism）是Transformer架构的核心组件，能够有效捕捉长序列内各元素间的依赖关系. 该机制通过计算输入序列中不同位置元素间的注意力得分，对其重要性进行精准建模，使模型在处理信息时能够聚焦于关键部分，从而显著提升对长序列数据的理解与处理能力.

在 MiniMindLM 模型中, Attention Block 包含以下机制和模块:

1. GQA
2. KV Cache
5. SwiGLU

- **GQA**

分组查询注意力 (Group Querey Attention, GQA) 是对多头自注意力机制的扩展, 通过提供计算效率和模型表达能力的灵活权衡, 实现了查询头的分组.

具体来说, GQA 中将 h 个查询头分为 G 组, 每组 包含 h / G 个查询头，并共享一个公共的键和值.

![img](./images/gqa.png)

GQA 相比传统的 MHA， 减少了键和值的数量，降低了计算量和内存开销，提高了推理速度.

- **KV Cache**

在语言模型生成文本的过程中，每生成一个新的 token，模型都需要计算注意力得分，以确定当前位置与之前所有位置的相关性.

比如以下内容：

1. *seq = \[tok1]:*

   attn_11 = softmax(Q1 * K1.T / sqrt(dim)) * V1
   
3. *seq = \[tok1, tok2]:*

   attn_11 = softmax(Q1 * K1.T / sqrt(dim)) * V1, attn_12 = 0 (masked)
   
   attn_21 = softmax(Q2 * K1.T / sqrt(dim)) * V1, attn_22 = softmax(Q2 * K2.T / sqrt(dim)) * V2
4. *seq = \[tok1, tok2, tok3]:*

   attn_11 = softmax(Q1 * K1.T / sqrt(dim)) * V1, attn_12 = 0 (masked), attn_13 = 0 (masked)
   
   attn_21 = softmax(Q2 * K1.T / sqrt(dim)) * V1, attn_22 = softmax(Q2 * K2.T / sqrt(dim)) * V2, attn_23 = 0 (masked)
   
   attn_31 = softmax(Q3 * K1.T / sqrt(dim)) * V1, attn_32 = softmax(Q3 * K2.T / sqrt(dim)) * V2, attn_33 = softmax(Q3 * K3.T / sqrt(dim)) * V3
5.  ··· ···

不难发现，大模型生成一个 token 后的注意力计算中，总会用到 token 序列的历史 KV 值，导致重复计算，KV Cache 的设计正是为了通过缓存历史 KV 值，节省计算开销.

KV Cache 能够有效压缩大模型推理时的显存占用.

- **SwiGLU**

SwiGLU 是一种在深度学习中用于神经网络架构的激活函数变体：

$$\text{SwiGLU}(x, W, V, b, c)=\text{Swish}_1(xW+b)\otimes(xV+c)$$

与传统的 ReLU 激活函数相比，SwiGLU 具有更好的平滑性和非线性表达能力，由于其门控机制，在处理信息筛选和流动方面有独特的优势.

In [ ]:
def repeat_kv(x: torch.Tensor, n_rep: int) -> torch.Tensor:
    """
    重复键值张量以适应注意力头的数量。

    参数:
    - x (torch.Tensor): 输入张量，形状为 (batch_size, seq_len, num_key_value_heads, head_dim)。
    - n_rep (int): 每个键值头需要重复的次数。

    返回:
    - torch.Tensor: 重复后的张量，形状为 (batch_size, seq_len, num_key_value_heads * n_rep, head_dim)。
    """
    bs, slen, num_key_value_heads, head_dim = x.shape
    if n_rep == 1:
        return x  # 如果不需要重复，直接返回原张量
    return (
        x[:, :, :, None, :]  # 在 num_key_value_heads 维度后插入一个新维度
        .expand(bs, slen, num_key_value_heads, n_rep, head_dim)  # 扩展新维度以重复 n_rep 次
        .reshape(bs, slen, num_key_value_heads * n_rep, head_dim)  # 重塑张量形状以合并重复维度
    )

class Attention(nn.Module):
    """
    自定义注意力模块，支持旋转位置编码和 Flash Attention。
    """
    
    def __init__(self, args: MiniMindConfig):
        super().__init__()
        # 初始化注意力模块的参数
        self.num_key_value_heads = args.num_attention_heads if args.num_key_value_heads is None else args.num_key_value_heads
        assert args.num_attention_heads % self.num_key_value_heads == 0  # 确保注意力头数量可以被键值头数量整除
        self.n_local_heads = args.num_attention_heads  # 本地注意力头数量
        self.n_local_kv_heads = self.num_key_value_heads  # 本地键值头数量
        self.n_rep = self.n_local_heads // self.n_local_kv_heads  # 每个键值头需要重复的次数
        self.head_dim = args.hidden_size // args.num_attention_heads  # 每个注意力头的维度
        # 定义线性投影层
        self.q_proj = nn.Linear(args.hidden_size, args.num_attention_heads * self.head_dim, bias=False)
        self.k_proj = nn.Linear(args.hidden_size, self.num_key_value_heads * self.head_dim, bias=False)
        self.v_proj = nn.Linear(args.hidden_size, self.num_key_value_heads * self.head_dim, bias=False)
        self.o_proj = nn.Linear(args.num_attention_heads * self.head_dim, args.hidden_size, bias=False)
        # 定义 Dropout 层
        self.attn_dropout = nn.Dropout(args.dropout)
        self.resid_dropout = nn.Dropout(args.dropout)
        self.dropout = args.dropout
        # 检查是否支持 Flash Attention
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention') and args.flash_attn

    def forward(self,
                x: torch.Tensor,
                position_embeddings: Tuple[torch.Tensor, torch.Tensor],  # 接收预计算的 cos 和 sin
                past_key_value: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,  # 历史键值缓存
                use_cache=False,  # 是否使用缓存
                attention_mask: Optional[torch.Tensor] = None):  # 注意力掩码
        bsz, seq_len, _ = x.shape
        # 计算 Query, Key, Value
        xq, xk, xv = self.q_proj(x), self.k_proj(x), self.v_proj(x)
        xq = xq.view(bsz, seq_len, self.n_local_heads, self.head_dim)
        xk = xk.view(bsz, seq_len, self.n_local_kv_heads, self.head_dim)
        xv = xv.view(bsz, seq_len, self.n_local_kv_heads, self.head_dim)

        # 应用旋转位置编码
        cos, sin = position_embeddings
        xq, xk = apply_rotary_pos_emb(xq, xk, cos[:seq_len], sin[:seq_len])

        # 处理 KV 缓存
        if past_key_value is not None:
            xk = torch.cat([past_key_value[0], xk], dim=1)  # 拼接历史键值
            xv = torch.cat([past_key_value[1], xv], dim=1)
        past_kv = (xk, xv) if use_cache else None  # 更新缓存

        # 重复键值以适应注意力头数量
        xq, xk, xv = (
            xq.transpose(1, 2),
            repeat_kv(xk, self.n_rep).transpose(1, 2),
            repeat_kv(xv, self.n_rep).transpose(1, 2)
        )

        # 使用 Flash Attention 或传统注意力计算
        if self.flash and seq_len != 1:
            dropout_p = self.dropout if self.training else 0.0
            attn_mask = None
            if attention_mask is not None:
                attn_mask = attention_mask.view(bsz, 1, 1, -1).expand(bsz, self.n_local_heads, seq_len, -1)
                attn_mask = attn_mask.bool() if attention_mask is not None else None

            # 使用 Flash Attention
            output = F.scaled_dot_product_attention(xq, xk, xv, attn_mask=attn_mask, dropout_p=dropout_p, is_causal=True)
        else:
            # 传统注意力计算
            scores = (xq @ xk.transpose(-2, -1)) / math.sqrt(self.head_dim)  # 计算注意力得分
            scores = scores + torch.triu(
                torch.full((seq_len, seq_len), float("-inf"), device=scores.device),
                diagonal=1
            ).unsqueeze(0).unsqueeze(0)  # 添加掩码

            if attention_mask is not None:
                extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
                extended_attention_mask = (1.0 - extended_attention_mask) * -1e9
                scores = scores + extended_attention_mask

            scores = F.softmax(scores.float(), dim=-1).type_as(xq)  # 归一化注意力得分
            scores = self.attn_dropout(scores)  # 应用 Dropout
            output = scores @ xv  # 计算注意力输出

        # 重塑输出形状并应用残差 Dropout
        output = output.transpose(1, 2).reshape(bsz, seq_len, -1)
        output = self.resid_dropout(self.o_proj(output))
        return output, past_kv  # 返回输出和更新后的缓存

同样的，我们假设一批 batch size = 4， seq len = 16 的 token 序列通过这个 Attention 块，在输入前，它的 input id 会被投影到 dim = 512 维.

In [ ]:
# 创建 Config，为了简单起见，我们设定 num_hidden_layers = 2
LMConfig_Dense = MiniMindConfig(num_hidden_layers=2)
# attn = Attention(LMConfig_Dense)
# x = torch.randn((4,  16,  512)) # (batch size, seq len, embed dim)
# pos_cis = precompute_pos_cis(64,  16) # (head dim, batch size) 其中 head dim = embed dim / num heads
# output,  past_kv = attn(x,  pos_cis=pos_cis,  use_cache=True)
# print(f'输入张量 x ：size = {x.shape}，RoPE 旋转角： size = {pos_cis.shape}')
# print(f'输出 output: size = {output.shape},  kv_cache 基本信息：size_key = {past_kv[0].shape}, size_value = {past_kv[1].shape}')
# del attn

### FeedForward Network

前馈神经网络接收来自注意力机制层的输出结果，随后对该输出执行进一步的线性变换. 通过这种方式，网络能够深入挖掘并捕获更为复杂、抽象的特征.

In [12]:
class FeedForward(nn.Module):
    def __init__(self, config: MiniMindConfig):
        super().__init__()
        # 如果未指定中间层大小，则根据隐藏层大小计算默认值
        if config.intermediate_size is None:
            intermediate_size = int(config.hidden_size * 8 / 3)
            # 将中间层大小调整为64的倍数
            config.intermediate_size = 64 * ((intermediate_size + 64 - 1) // 64)
        
        # 定义前馈网络的线性层
        self.gate_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)  # 输入到中间层的投影
        self.down_proj = nn.Linear(config.intermediate_size, config.hidden_size, bias=False)  # 中间层到输出的投影
        self.up_proj = nn.Linear(config.hidden_size, config.intermediate_size, bias=False)  # 输入到中间层的另一路投影
        self.dropout = nn.Dropout(config.dropout)  # Dropout层
        self.act_fn = ACT2FN[config.hidden_act]  # 激活函数

    def forward(self, x):
        # 前向传播：激活函数作用于 gate_proj 的输出，并与 up_proj 的输出相乘，最后通过 down_proj 和 Dropout
        return self.dropout(self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x)))

设置输入 x 为 batch size = 4，seq len = 16 的 token 序列投影向量，观察 x 在 MiniMind Block 的前向传播过程.

In [13]:
ffn = FeedForward(LMConfig_Dense)
x = torch.randn((4,  16,  512))
output = ffn(x)
print(f'给定输入 x: size = {x.shape} 下的输出 output：size = {output.shape}')

给定输入 x: size = torch.Size([4, 16, 512]) 下的输出 output：size = torch.Size([4, 16, 512])


In [14]:
del ffn

这就是 Transformer 结构的精妙之处，张量在模型内部进行了各种复杂的投影变形，但是输入输出的张量形状不发生改变!

### MiniMind Block

到目前为止, , 已经完成了 Attention Layer 和 FeedForward Layer 的构建, 所有必须的组件都已经具备, 我们着手构建一个 MiniMind Block

In [15]:
class MiniMindBlock(nn.Module):
    def __init__(self, layer_id: int, config: MiniMindConfig):
        super().__init__()
        # 初始化参数
        self.num_attention_heads = config.num_attention_heads  # 注意力头数量
        self.hidden_size = config.hidden_size  # 隐藏层大小
        self.head_dim = config.hidden_size // config.num_attention_heads  # 每个注意力头的维度
        self.self_attn = Attention(config)  # 自注意力模块

        self.layer_id = layer_id  # 当前层的编号
        self.input_layernorm = RMSNorm(config.hidden_size, eps=config.rms_norm_eps)  # 输入层归一化
        self.post_attention_layernorm = RMSNorm(config.hidden_size, eps=config.rms_norm_eps)  # 注意力后的归一化
        self.mlp = FeedForward(config)  # 前馈神经网络

    def forward(self, hidden_states, position_embeddings, past_key_value=None, use_cache=False, attention_mask=None):
        # 残差连接
        residual = hidden_states
        # 自注意力计算
        hidden_states, present_key_value = self.self_attn(
            self.input_layernorm(hidden_states),  # 对输入进行归一化
            position_embeddings,  # 位置编码
            past_key_value,  # 历史键值缓存
            use_cache,  # 是否使用缓存
            attention_mask  # 注意力掩码
        )
        hidden_states += residual  # 添加残差连接
        # 前馈神经网络计算并添加残差连接
        hidden_states = hidden_states + self.mlp(self.post_attention_layernorm(hidden_states))
        return hidden_states, present_key_value  # 返回更新后的隐藏状态和键值缓存

我们依然设置输入 x 为 batch size = 4，seq len = 16 的 token 序列投影向量，观察 x 在 MiniMind Block 的前向传播过程.

In [ ]:
# miniblock = MiniMindBlock(1,  LMConfig_Dense)
# x = torch.randn((4,  16,  512))
# pos_cis = precompute_pos_cis(64,  16)
# out,  past_kv = miniblock(x,  pos_cis,  use_cache=True)
# print(f'输出 output 信息: size = {out.shape}\n该 Block 维护的 KV Cache 信息：size_key =  {past_kv[0].shape}, size_value = {past_kv[1].shape}')
# del miniblock

输出 output 信息: size = torch.Size([4, 16, 512])
该 Block 维护的 KV Cache 信息：size_key =  torch.Size([4, 16, 2, 64]), size_value = torch.Size([4, 16, 2, 64])


### MiniMindLM (Dense)

以 MiniMind Block 为基本组件, 我们对 MiniMindLM 进行最后组装！

In [18]:
class MiniMindModel(nn.Module):
    def __init__(self, config: MiniMindConfig):
        super().__init__()
        self.config = config
        self.vocab_size, self.num_hidden_layers = config.vocab_size, config.num_hidden_layers
        # 定义词嵌入层
        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size)
        # 定义 Dropout 层
        self.dropout = nn.Dropout(config.dropout)
        # 定义多个 MiniMindBlock 层
        self.layers = nn.ModuleList([MiniMindBlock(l, config) for l in range(self.num_hidden_layers)])
        # 定义 RMSNorm 层
        self.norm = RMSNorm(config.hidden_size, eps=config.rms_norm_eps)

        # 预计算旋转位置编码的频率和相位
        freqs_cos, freqs_sin = precompute_freqs_cis(dim=config.hidden_size // config.num_attention_heads,
                                                    end=config.max_position_embeddings, theta=config.rope_theta)
        # 注册为模型的缓冲区
        self.register_buffer("freqs_cos", freqs_cos, persistent=False)
        self.register_buffer("freqs_sin", freqs_sin, persistent=False)

    def forward(self,
                input_ids: Optional[torch.Tensor] = None,  # 输入的 token ID
                attention_mask: Optional[torch.Tensor] = None,  # 注意力掩码
                past_key_values: Optional[List[Tuple[torch.Tensor, torch.Tensor]]] = None,  # 历史键值缓存
                use_cache: bool = False,  # 是否使用缓存
                **kwargs):
        # 获取批次大小和序列长度
        batch_size, seq_length = input_ids.shape
        # 如果没有提供 past_key_values，则初始化为空
        past_key_values = past_key_values or [None] * len(self.layers)
        # 获取起始位置
        start_pos = past_key_values[0][0].shape[1] if past_key_values[0] is not None else 0

        # 嵌入输入的 token 并应用 Dropout
        hidden_states = self.dropout(self.embed_tokens(input_ids))

        # 获取当前位置的旋转位置编码
        position_embeddings = (
            self.freqs_cos[start_pos:start_pos + seq_length],
            self.freqs_sin[start_pos:start_pos + seq_length]
        )

        presents = []  # 用于存储每一层的键值缓存
        for layer_idx, (layer, past_key_value) in enumerate(zip(self.layers, past_key_values)):
            # 通过每一层的前向传播
            hidden_states, present = layer(
                hidden_states,
                position_embeddings,  # 位置编码
                past_key_value=past_key_value,  # 历史键值缓存
                use_cache=use_cache,  # 是否使用缓存
                attention_mask=attention_mask  # 注意力掩码
            )
            presents.append(present)  # 保存当前层的键值缓存

        # 应用 RMSNorm 层
        hidden_states = self.norm(hidden_states)

        # MoE 辅助损失，在 Dense 模型中不纳入考虑
        aux_loss = 0

        # 返回隐藏状态、键值缓存和辅助损失
        return hidden_states, presents, aux_loss

接下来，我们设置一条长度为 4 的 token 序列，使用 MiniMindLM 对其执行一次前向传播，观察传播过程与返回值.

In [ ]:
MiniMind_Dense = MiniMindModel(LMConfig_Dense)
input_ids = torch.Tensor([1,  3,  5,  7]).long().reshape(1,  4)
OUT = MiniMind_Dense(input_ids,  use_cache=True)
print(f'返回 logits：size = {OUT[0].shape}, 返回 aux_loss: {OUT[2]},  返回 KV Cache List： len = {len(OUT[1])}')
del MiniMind_Dense

返回 logits：size = torch.Size([1, 4, 512]), 返回 aux_loss: 0,  返回 KV Cache List： len = 2


## Minimind MoE Model

作者提供了 MiniMind MoE Model 的可视化.

![image](../images/LLM-structure-moe.png)

可以看到，Dense Model 和 MoE Model 的差异在于 FFN 层. MoE 模型将稠密连接的 FFN 层置换为 M x Expert 层，每次前向传播时只激活部分 Expert.

其组成可以分为以下部分：

- Experts: MoE 架构单元，每个专家本质上是一个独立的神经网络模块，负责处理特定类型或范围的数据.
- Router： 控制信息流动，决定每次前向传播激活的 Experts 模块以及输入数据在这些模块的分配组合.

在 MoE 网络中，为了平衡专家的重要性，我们需要关注路由，它是决定在特定时间选择哪些专家的重要组件.

### 辅助损失

为了让训练过程中实现专家的更均匀分布，辅助损失（又称负载均衡损失）被添加到网络的常规损失中，它增加了一个约束，迫使专家具有相等的重要性.

假设有输入序列 \[What is Mixture of Experts], *Prob* (·) 表示每一个 token 激活的专家概率分布:

- **Step 1**：在整个批次中对每个专家的路由值进行求和.

   $$Importance \, per \, token = Prob (What) + Prob (is) + Prob (Mixture) + Prob (of) + Prob (Experts)$$

   这个指标反映了 batch 维度上每个专家的重要性分数.

- **Step 2**: 计算变异系数

   我们希望专家之间的重要性尽可能靠近，为了衡量专家得分之间的差异程度，引入变异系数指标（Coefficient Variation, CV）

   $$Coeifficient \, Variation (CV) = \dfrac{standard \, deviation (\sigma)}{mean(\mu)}$$

   如果专家的重要性分数相似，变异系数会降到很低（这是我们期望的）

- **Step 3**: 计算负载均衡损失

   $$uxiliary Loss = \alpha * CV$$

   其中 $\alpha$ 是缩放系数.

## MoE Gate

MoE 门控单元决定每次前向传播激活的 Experts 模块及其权重，同时计算 MoE 辅助损失.

In [20]:
class MoEGate(nn.Module):
    def __init__(self, config: LMConfig):
        super().__init__()
        self.config = config
        self.top_k = config.num_experts_per_tok
        self.n_routed_experts = config.n_routed_experts # 总专家数量

        self.scoring_func = config.scoring_func # 评分函数
        self.alpha = config.aux_loss_alpha # 辅助损失的 alpha 参数
        self.seq_aux = config.seq_aux # 是否启用序列辅助损失

        self.norm_topk_prob = config.norm_topk_prob
        self.gating_dim = config.dim
        self.weight = nn.Parameter(torch.empty((self.n_routed_experts, self.gating_dim))) # 混合专家层 -> embeding dim 层
        self.reset_parameter()

    def reset_parameter(self):
        import torch.nn.init as init
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))

    def forward(self, hidden_states):
        bsz, seq_len, h = hidden_states.shape
        hidden_states = hidden_states.view(-1, h) # (total_tokens, hidden_dim) 其中 total_tokens = bsz * seq_len
        logits = F.linear(hidden_states, self.weight, None) # logits = hidden_states · self.weight.T
        # print(f'shape of logits computed: {logits.shape}') # (total_tokens, n_routed_experts)

        if self.scoring_func == 'softmax':
            scores = logits.softmax(dim=-1)
        else:
            raise NotImplementedError(f'insupportable scoring function for MoE gating: {self.scoring_func}')

        topk_weight, topk_idx = torch.topk(scores, k=self.top_k, dim=-1, sorted=False)
        if self.top_k > 1 and self.norm_topk_prob: # prob 归一化
            denominator = topk_weight.sum(dim=-1, keepdim=True) + 1e20
            topk_weight = topk_weight / denominator

        if self.training and self.alpha > 0.0:
            scores_for_aux = scores
            aux_topk = self.top_k
            topk_idx_for_aux_loss = topk_idx.view(bsz, -1) # (bsz, seq_len * num_experts_per_tok)
            if self.seq_aux:
                ################# 对所有批次计算 Experts 重要性得分 #################
                scores_for_seq_aux = scores_for_aux.view(bsz, seq_len, -1)
                ce = torch.zeros(bsz, self.n_routed_experts, device=hidden_states.device)
                # 根据索引将 torch.ones 累加到 ce 中，并执行归一化
                ce.scatter_add_(1, topk_idx_for_aux_loss,
                                torch.ones(bsz, seq_len * aux_topk, device=hidden_states.device)).div_(
                    seq_len * aux_topk / self.n_routed_experts)
                ######################### 计算负载均衡损失 ##########################
                aux_loss = (ce * scores_for_seq_aux.mean(dim=1)).sum(dim=1).mean() * self.alpha
            else:
                mask_ce = F.one_hot(topk_idx_for_aux_loss.view(-1), num_classes=self.n_routed_experts)
                ce = mask_ce.float().mean(0)
                Pi = scores_for_aux.mean(0)
                fi = ce * self.n_routed_experts
                aux_loss = (Pi * fi).sum() * self.alpha
        else:
            aux_loss = 0
        return topk_idx, topk_weight, aux_loss

接下来，我们设置一个 batch size = 4, seq len =16, emb dim = 512 的输入向量，在 MoE 门控单元前向传播进行观察

In [21]:
# 重新创建一个 LMConfig 使得 use MoE 为 True
LMConfig_MoE = LMConfig(n_layers=2, use_moe=True)

In [22]:
gate = MoEGate(LMConfig_MoE)
hidden_states = torch.randn((4, 16, 512))
topk_idx, topk_weight, aux_loss = gate(hidden_states)
print(f'shape: topk_idx = {topk_idx.shape}, topk_weight = {topk_weight.shape}, aux_loss = {aux_loss}')
print(f'token 0 选择的专家：idx = {topk_idx[0]}, weight = {topk_weight[0]}')
print(f'辅助损失：aux_loss = {aux_loss}')

shape: topk_idx = torch.Size([64, 2]), topk_weight = torch.Size([64, 2]), aux_loss = 0.10163114219903946
token 0 选择的专家：idx = tensor([2, 0]), weight = tensor([5.8955e-21, 1.9281e-21], grad_fn=<SelectBackward0>)
辅助损失：aux_loss = 0.10163114219903946


In [23]:
del gate

### MoE Feed Forward NetWork

完成 MoE 门控单元的设计后，我们可以对 MoE 前向传播网络进行重新设计.

In [24]:
class MoEFeedForward(nn.Module):
    def __init__(self, config: LMConfig):
        super().__init__()
        self.config = config
        # 可选专家层
        self.experts = nn.ModuleList([
            FeedForward(config)
            for _ in range(config.n_routed_experts)
        ])
        self.gate = MoEGate(config)
        # 共享专家层d
        if config.n_shared_experts is not None:
            self.shared_experts = FeedForward(config)

    def forward(self, x):
        identity = x
        orig_shape = x.shape
        bsz, seq_len, _ = x.shape
        # 使用门控机制选择专家
        topk_idx, topk_weight, aux_loss = self.gate(x)
        x = x.view(-1, x.shape[-1])
        flat_topk_idx = topk_idx.view(-1)
        if self.training:
            # 训练模式下，重复输入数据
            x = x.repeat_interleave(self.config.num_experts_per_tok, dim=0)
            y = torch.empty_like(x, dtype=torch.float16)
            # 将输入数据中选择第 i 个专家的部分输入到该专家网络中进行处理，并将输出结果存储到 y 中对应位置的元素
            for i, expert in enumerate(self.experts):
                y[flat_topk_idx == i] = expert(x[flat_topk_idx == i]).to(y.dtype)  #  确保类型一致
            y = (y.view(*topk_weight.shape, -1) * topk_weight.unsqueeze(-1)).sum(dim=1) # 加权求和
            y = y.view(*orig_shape) # 恢复原始形状
        else:
            # 推理模式下，只选择最优专家
            y = self.moe_infer(x, flat_topk_idx, topk_weight.view(-1, 1)).view(*orig_shape)
        if self.config.n_shared_experts is not None:
            y = y + self.shared_experts(identity)
        self.aux_loss = aux_loss
        return y

    @torch.no_grad()
    def moe_infer(self, x, flat_expert_indices, flat_expert_weights):
        expert_cache = torch.zeros_like(x)
        idxs = flat_expert_indices.argsort() # [exp_tok_11, exp_tok_12, exp_tok_21, exp_tok_22] -> idxs[2 * token_idxs] = moe_expert_name
        tokens_per_expert = flat_expert_indices.bincount().cpu().numpy().cumsum(0) #  [1, 3, 1, 2] -> bincount -> [0, 2, 1, 1] -> cumsum -> [0, 2, 3, 4]
        token_idxs = idxs // self.config.num_experts_per_tok
        # 例如当tokens_per_expert=[6, 15, 20, 26, 33, 38, 46, 52]
        # 当token_idxs=[3, 7, 19, 21, 24, 25,  4,  5,  6, 10, 11, 12...]
        # 意味着当token_idxs[:6] -> [3,  7, 19, 21, 24, 25,  4]位置的token都由专家0处理，token_idxs[6:15]位置的token都由专家1处理......
        for i, end_idx in enumerate(tokens_per_expert):
            start_idx = 0 if i == 0 else tokens_per_expert[i - 1]
            if start_idx == end_idx:
                continue
            expert = self.experts[i]
            exp_token_idx = token_idxs[start_idx:end_idx]
            expert_tokens = x[exp_token_idx]
            expert_out = expert(expert_tokens).to(expert_cache.dtype)
            expert_out.mul_(flat_expert_weights[idxs[start_idx:end_idx]]) # 专家网络输出加权
            # 使用 scatter_add_ 进行 sum 操作
            expert_cache.scatter_add_(0, exp_token_idx.view(-1, 1).repeat(1, x.shape[-1]), expert_out)

        return expert_cache

接下来，我们设置一个 batch size = 4, seq len =16, emb dim = 512 的输入向量，在 MoE 门控单元前向传播进行观察

In [25]:
moe_ffn = MoEFeedForward(LMConfig_MoE).eval()
x = torch.randn((4, 16, 512))
output = moe_ffn(x)
print(f'输出张量：shape = {output.shape}, 辅助损失：aux_loss = {moe_ffn.aux_loss}')

输出张量：shape = torch.Size([4, 16, 512]), 辅助损失：aux_loss = 0


In [26]:
del moe_ffn

之前提到过，MiniMind MoE 和 MiniMind Dense 的最大差别在于 FFN 模块的不同，我们可以对之前声明的 MiniMindBlock 进行继承，添加 MoE FFN 选项.

In [27]:
# DM stands for Dense & Moe
class MiniMindBlock_DM(MiniMindBlock):
    def __init__(self, layer_id: int, config: LMConfig):
        super().__init__(layer_id, config)
        self.feed_forward = FeedForward(config) if not config.use_moe else MoEFeedForward(config)

In [28]:
# 检查继承是否正常

miniblock_dm = MiniMindBlock_DM(1, LMConfig_MoE)
print(f'类变量属性检查：layer_id = {miniblock_dm.layer_id}, 类函数属性检查：forward func = {miniblock_dm.forward}')

类变量属性检查：layer_id = 1, 类函数属性检查：forward func = <bound method MiniMindBlock.forward of MiniMindBlock_DM(
  (attention): Attention(
    (wq): Linear(in_features=512, out_features=512, bias=False)
    (wk): Linear(in_features=512, out_features=128, bias=False)
    (wv): Linear(in_features=512, out_features=128, bias=False)
    (wo): Linear(in_features=512, out_features=512, bias=False)
    (attn_dropout): Dropout(p=0.0, inplace=False)
    (resid_dropout): Dropout(p=0.0, inplace=False)
  )
  (attention_norm): RMSNorm()
  (ffn_norm): RMSNorm()
  (feed_forward): MoEFeedForward(
    (experts): ModuleList(
      (0-3): 4 x FeedForward(
        (w1): Linear(in_features=512, out_features=1408, bias=False)
        (w2): Linear(in_features=1408, out_features=512, bias=False)
        (w3): Linear(in_features=512, out_features=1408, bias=False)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (gate): MoEGate()
    (shared_experts): FeedForward(
      (w1): Linear(in_features=512, out

我们仍然设置一个 batch size = 4, seq len =16, emb dim = 512 的输入向量，在 MoE 门控单元前向传播进行观察.

In [29]:
miniblock_dm.eval() # 冻结参数，直接 moe infer
x = torch.randn((4, 16, 512))
pos_cis = precompute_pos_cis(64, 16)
output, past_kv = miniblock_dm(x, pos_cis, use_cache=True)
print(f'输出张量：shape = {output.shape}, KV Cache: shape Key = {past_kv[0].shape}, shape Value = {past_kv[1].shape}')
print(f'辅助损失：aux_loss = {miniblock_dm.feed_forward.aux_loss}')

输出张量：shape = torch.Size([4, 16, 512]), KV Cache: shape Key = torch.Size([4, 16, 2, 64]), shape Value = torch.Size([4, 16, 2, 64])
辅助损失：aux_loss = 0


In [30]:
del miniblock_dm

如此，我们便完成了包含 MoE 和 Dense 两种 FFN 选项的 MiniMind Block 的定义，我们对此前声明的 MiniMindLM 作继承修改，使其具备 MoE 架构.

In [31]:
class MiniMindLM_DM(MiniMindLM):
    def __init__(self, params: LMConfig = None):
        super().__init__(params)
        structure = 'MoE' if params.use_moe else 'Dense'
        print(f'Initializing MiniMind {structure} Model...\n')
        self.layers = nn.ModuleList([MiniMindBlock_DM(l, params) for l in range(self.n_layers)])

In [32]:
# 检查一下
a = MiniMindLM_DM(LMConfig_Dense)
b = MiniMindLM_DM(LMConfig_MoE)
del a, b

Initializing MiniMind Dense Model...

Initializing MiniMind MoE Model...



接下来，我们设置一条长度为 4 的 token 序列，使用 MiniMindLM 对其执行一次前向传播，观察传播过程与返回值.

In [33]:
MiniMind_MoE = MiniMindLM_DM(LMConfig_MoE)
input_ids = torch.Tensor([1,  3,  5,  7]).long().reshape(1,  4)
OUT = MiniMind_MoE(input_ids,  use_cache=True)
print(f'返回 logits：size = {OUT.logits.shape}, 返回 aux_loss: {OUT.aux_loss},  返回 KV Cache List： len = {len(OUT.past_key_value)}')

Initializing MiniMind MoE Model...

====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

返回 logits：size = torch.Size([1, 4, 6400]), 返回 aux_loss: 0,  返回 KV Cache List： len = 2


In [34]:
# 我们让 MiniMind 根据我们设计的 四个输入 token 生成输出
out = MiniMind_MoE.generate(input_ids,  max_new_tokens=8,  use_cache=True)
print(f'生成结果：{out}')

gernerating new token: idx = 4
====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 4, 2, 64]), size_cache_v = torch.Size([1, 4, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

gernerating new token: idx = 5
====> forward propagation started, num_minimind_blocks = 2
------------> entering minimind block: id = 0
<------------ finished, size_cache_k = torch.Size([1, 1, 2, 64]), size_cache_v = torch.Size([1, 1, 2, 64])
------------> entering minimind block: id = 1
<------------ finished, size_cache_k = torch.Size([1, 1, 2, 64]), size_cache_v = torch.Size([1, 1, 2, 64])
<==== forward propagation completed, num_kv_cache = 2

gernerating new token: idx = 6
====> forward propagation started, num_minimind_blocks = 2
--------

In [35]:
del MiniMind_MoE

## 参考资料

- [十分钟读懂旋转编码（RoPE）](https://www.zhihu.com/tardis/zm/art/647109286?source_id=1003)
- [混合专家模型 MoE 的全面指南](https://blog.csdn.net/Code1994/article/details/145209710#:~:text=%E4%B8%BA%E4%BA%86%E5%9C%A8%E8%AE%AD%E7%BB%83%E8%BF%87%E7%A8%8B%E4%B8%AD%E5%AE%9E%E7%8E%B0%E4%B8%93%E5%AE%B6%E7%9A%84%E6%9B%B4%E5%9D%87%E5%8C%80%E5%88%86%E5%B8%83%EF%BC%8C%E8%BE%85%E5%8A%A9%E6%8D%9F%E5%A4%B1%EF%BC%88%E4%B9%9F%E7%A7%B0%E4%B8%BA%E8%B4%9F%E8%BD%BD%E5%9D%87%E8%A1%A1%E6%8D%9F%E5%A4%B1%EF%BC%89%E8%A2%AB%E6%B7%BB%E5%8A%A0%E5%88%B0%E4%BA%86%E7%BD%91%E7%BB%9C%E7%9A%84%E5%B8%B8%E8%A7%84%E6%8D%9F%E5%A4%B1%E4%B8%AD%E3%80%82%20%E5%AE%83%E5%A2%9E%E5%8A%A0%E4%BA%86%E4%B8%80%E4%B8%AA%E7%BA%A6%E6%9D%9F%EF%BC%8C%E8%BF%AB%E4%BD%BF%E4%B8%93%E5%AE%B6%E5%85%B7%E6%9C%89%E7%9B%B8%E7%AD%89%E7%9A%84%E9%87%8D%E8%A6%81%E6%80%A7%E3%80%82,%E8%BF%99%E4%B8%AA%E8%BE%85%E5%8A%A9%E6%8D%9F%E5%A4%B1%E7%9A%84%E7%AC%AC%E4%B8%80%E4%B8%AA%E7%BB%84%E6%88%90%E9%83%A8%E5%88%86%E6%98%AF%E5%9C%A8%E6%95%B4%E4%B8%AA%E6%89%B9%E6%AC%A1%E4%B8%AD%E5%AF%B9%E6%AF%8F%E4%B8%AA%E4%B8%93%E5%AE%B6%E7%9A%84%E8%B7%AF%E7%94%B1%E5%99%A8%E5%80%BC%E8%BF%9B%E8%A1%8C%E6%B1%82%E5%92%8C%EF%BC%9A%20%E8%BF%99%E4%B8%BA%E6%88%91%E4%BB%AC%E6%8F%90%E4%BE%9B%E4%BA%86%E6%AF%8F%E4%B8%AA%E4%B8%93%E5%AE%B6%E7%9A%84%E9%87%8D%E8%A6%81%E6%80%A7%E5%88%86%E6%95%B0%EF%BC%8C%E8%BF%99%E4%BA%9B%E5%88%86%E6%95%B0%E8%A1%A8%E7%A4%BA%E6%97%A0%E8%AE%BA%E8%BE%93%E5%85%A5%E6%98%AF%E4%BB%80%E4%B9%88%EF%BC%8C%E7%BB%99%E5%AE%9A%E4%B8%93%E5%AE%B6%E8%A2%AB%E9%80%89%E4%B8%AD%E7%9A%84%E5%8F%AF%E8%83%BD%E6%80%A7%E3%80%82%20%E6%88%91%E4%BB%AC%E5%8F%AF%E4%BB%A5%E5%88%A9%E7%94%A8%E8%BF%99%E4%BA%9B%E5%88%86%E6%95%B0%E6%9D%A5%E8%AE%A1%E7%AE%97%E5%8F%98%E5%BC%82%E7%B3%BB%E6%95%B0%EF%BC%88CV%EF%BC%89%EF%BC%8C%E5%AE%83%E5%91%8A%E8%AF%89%E6%88%91%E4%BB%AC%E4%B8%93%E5%AE%B6%E4%B9%8B%E9%97%B4%E7%9A%84%E9%87%8D%E8%A6%81%E6%80%A7%E5%88%86%E6%95%B0%E7%9A%84%E5%B7%AE%E5%BC%82%E7%A8%8B%E5%BA%A6%E3%80%82)